In [23]:
import os
import json
import yaml
from glob import glob

from shutil import copyfile

import matplotlib.pyplot as plt
from ultralytics import YOLO

In [25]:
IMG_BASE_DIR = "../data/ai03-level1-project/train_images"
ANNOTATION_DIR = "../data/ai03-level1-project/train_annotations"
ann_folder_list = os.listdir(ANNOTATION_DIR)
ann_folder_list = sorted(ann_folder_list)

# category_id_map: { "19860": "노바스크정 5mg", ... }
with open("../Project/data_csv/mappings.json", "r") as f:
    mappings = json.load(f)

# YOLO class index -> class name
with open("../data/ai03-level1-project/data.yaml", "r") as f:
    yaml_data = yaml.safe_load(f)
yolo_classes = yaml_data["names"]

# class name -> category_id
name_to_catid = {v: int(k) for k, v in mappings["category_id_map"].items()}

# YOLO index → category_id
yolo_idx_to_catid = {i: name_to_catid[name] for i, name in enumerate(yolo_classes)}

model = YOLO("../best.pt")

In [28]:
def make_loss_file():
  model.eval()
  
  # train_annotations/K-003544-012247-016548-027926_json
  for folder_name in ann_folder_list:
    # [003544, 012247, 016548, 027926]
    keys = (folder_name.split("_json")[0]).split("-")[1:]

    # train_annotations/K-003544-012247-016548-027926_json/K-003544
    target_path = os.path.join(ANNOTATION_DIR, folder_name)

    # [K-003544, K-012247, K-016548, K-027926]
    target_folders = os.listdir(target_path)

    if len(keys) != len(target_folders):
      # 폴더 자체가 없는경우는 메모만
      print(f"len(keys) != len(target_folders) {folder_name}")

    for target_folder_name in target_folders:
      # 003544
      if target_folder_name.split("-")[1] in keys:
        path3 = os.path.join(target_path, target_folder_name)

        # [.json]
        json_files = os.listdir(path3)
        if len(json_files) == 0:
          continue
        elif len(json_files) < 3:
          try:
            current_keys = [(i.split("0_2_0_2_")[1]).split("_000_")[0] for i in json_files]
          except:
            print("except key: ", folder_name)
            current_keys = [(i.split("0_2_1_2_")[1]).split("_000_")[0] for i in json_files]

          for current_key in ["70", "75", "90"]:
            if current_key not in current_keys:
              origin_filepath = os.path.join(path3, json_files[0])
              copy_filename = f"{(json_files[0].split('_000_')[0][:-2])}{current_key}_000_200"
              copy_filepath = os.path.join(path3, f"{copy_filename}.json")
              
			  # 이미지가 없으면 패스
              if not os.path.exists(f"{IMG_BASE_DIR}/{copy_filename}.png"):
                continue
              
              copyfile(origin_filepath, copy_filepath)

              with open(copy_filepath, "r", encoding="utf-8") as f:
                ann = json.load(f)
              # 값 수정
              ann["images"][0]["file_name"] = f"{copy_filename}.png"
              ann["images"][0]["imgfile"] = f"{copy_filename}.png"
              
			  # image_id 값 찾기
              image_id = None
              for i in target_folders:
                if i != target_folder_name:
                  ref_file = os.path.join(target_path, i, f"{copy_filename}.json")
                  if os.path.exists(ref_file):
                    with open(ref_file, "r", encoding="utf-8") as f:
                      ref_ann = json.load(f)
                      
                      image_id = ref_ann["annotations"][0]["image_id"]
                      break
              if image_id is None:
                print("=========", path3)
                return
              
              ann["images"][0]["id"] = image_id
              ann["annotations"][0]["image_id"] = image_id
			
              # 추론
              results = model(f"{IMG_BASE_DIR}/{copy_filename}.png", conf=0.5)
              for i, box in enumerate(results[0].boxes):
                class_id = int(box.cls[0])
                conf = float(box.conf[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                if ann["annotations"][0]["category_id"] == yolo_idx_to_catid.get(class_id):
                  ann["annotations"][0]["bbox"] = [x1, y1, x2-x1, y2-y1]
              
			  # JSON 파일 덮어쓰기
              with open(copy_filepath, "w", encoding="utf-8") as f:
                  json.dump(ann, f, indent=4, ensure_ascii=False)
                  

results = make_loss_file()

len(keys) != len(target_folders) K-002483-003743-012081-022627_json
len(keys) != len(target_folders) K-002483-005094-012081-019552_json
len(keys) != len(target_folders) K-002483-012081-023223-025438_json
len(keys) != len(target_folders) K-003351-019232-020238_json
except key:  K-003351-019232-029667_json
except key:  K-003351-019232-029667_json
except key:  K-003351-019232-029667_json
len(keys) != len(target_folders) K-003483-016232-020877-022347_json
len(keys) != len(target_folders) K-003483-016232-022347-027653_json
len(keys) != len(target_folders) K-003483-016232-027653-031885_json
len(keys) != len(target_folders) K-003483-016232-027653-034597_json
len(keys) != len(target_folders) K-003483-016262-020877-022347_json
len(keys) != len(target_folders) K-003483-016262-027733-031885_json
len(keys) != len(target_folders) K-003483-019861-028763-029667_json
len(keys) != len(target_folders) K-003483-025469-034597-035206_json
len(keys) != len(target_folders) K-003483-027653-031885-036637_json


In [ ]:
def find_outbindding_box():
  model.eval()

  image_files = os.listdir(IMG_BASE_DIR)
  
  for image_file in image_files:
    # 어노테이션의 바운딩박스
    key_1, key_2 = image_file.split("_")
    target_dir = os.path.join(ANNOTATION_DIR, f"{key_1}_json")
    json_list = glob(f"{target_dir}/*.json")


	# 모델의 바운딩박스
    # results = model(f"{IMG_BASE_DIR}/{image_file}", conf=0.5)
    # for i, box in enumerate(results[0].boxes):
    #   class_id = int(box.cls[0])
    #   conf = float(box.conf[0])
    #   x1, y1, x2, y2 = map(int, box.xyxy[0])
    pass
  pass

In [ ]:
result = results[0]  # 단일 이미지 추론 결과

for i, box in enumerate(result.boxes):
    class_id = int(box.cls[0])
    conf = float(box.conf[0])
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    print(f"[{i}] class={yolo_idx_to_catid.get(class_id)}, conf={conf:.2f}, box=[{x1:.1f}, {y1:.1f}, {x2:.1f}, {y2:.1f}]")

img_with_boxes = result.plot()  # numpy array
plt.imshow(img_with_boxes)
plt.axis("off")
plt.show()

TypeError: 'NoneType' object is not subscriptable